In [3]:
import tensorflow as tf
print(tf.__version__)


ModuleNotFoundError: No module named 'tensorflow.python'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load datasets (use your local folders)
train_dataset = datasets.ImageFolder('C:\\Users\\hp\\Downloads', transform=transform)
val_dataset = datasets.ImageFolder('C:\\Users\\hp\\Downloads', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Define CNN Model
class PlantCNN(nn.Module):
    def __init__(self):
        super(PlantCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*32*32, 128), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)  # 2 classes: healthy/infected
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

model = PlantCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


FileNotFoundError: Found no valid file for the classes Data Science Project, Intenship, Intenship - Copy, New folder, archive, archive (1). Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
for epoch in range(5):  # Short run for demo
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


In [ ]:
model.eval()
correct = total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


In [ ]:
import numpy as np
from scipy import stats

# Simulated validation accuracy from 10 different runs
A_accuracies = np.array([0.85, 0.84, 0.86, 0.83, 0.85, 0.84, 0.86, 0.82, 0.84, 0.85])
B_accuracies = np.array([0.88, 0.87, 0.89, 0.86, 0.88, 0.89, 0.87, 0.88, 0.90, 0.87])

# Mean and std
print("Mean A:", A_accuracies.mean())
print("Mean B:", B_accuracies.mean())


In [ ]:
import matplotlib.pyplot as plt

plt.hist(A_accuracies, alpha=0.5, label='Model A')
plt.hist(B_accuracies, alpha=0.5, label='Model B')
plt.legend()
plt.title('Validation Accuracy Distributions')
plt.xlabel('Accuracy')
plt.ylabel('Frequency')
plt.show()


In [ ]:
t_stat, p_value = stats.ttest_ind(B_accuracies, A_accuracies, equal_var=False)
print(f"T-statistic: {t_stat:.3f}, P-value: {p_value:.4f}")


In [ ]:
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis — Model B performs significantly better.")
else:
    print("Fail to reject the null — No significant difference in model performance.")
